# 프롬노웨어 크롤링
- 남녀구분 잘 안됨
- 카테고리별로 크롤링 진행
- 한 페이지당 100개 품목


In [1]:
import time
import json
from collections import defaultdict

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
class create_dict(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value

In [9]:
# 1페이지에 존재하는 품목들을 가져오는 함수
def get_product_list():
    css = '#contents > div.xans-element-.xans-product.xans-product-normalpackage > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    product_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    products = product_list.find_elements_by_xpath(xpath)
    return products

# 품목 카테고리 가져오는 함수
def get_product_category():
    css = '#contents > div.xans-element-.xans-product.xans-product-menupackage > div > h2'
    product_category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_category = product_category.text
    print(product_category)
    return product_category

# 품목을 클릭하는 함수
def click_product(iter):
    css1 = '#contents > div.xans-element-.xans-product.xans-product-normalpackage > div.xans-element-.xans-product.xans-product-listnormal > ul'
    xpath = '//*[starts-with(@id, "anchorBoxId_")]'
    css2 = 'div > a'
    # 드라이버 재정의
    product_list = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css1)))
    products = product_list.find_elements_by_xpath(xpath)
    # url
    # product_link = WebDriverWait(products[iter], 3).until(EC.presence_of_element_located((By.XPATH, xpath)))
    # link = items[N].find_element_by_css_selector('div > a')
    product_link = products[iter].find_element_by_css_selector('div > a').get_attribute('href')
    # 박스 클릭
    product_box = WebDriverWait(products[iter], 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, css2)))
    product_box.click()
    print(f'link:{product_link}')
    time.sleep(1)
    return product_link

# 품목 이름을 가져오는 함수
def get_product_name():
    class_name = 'ndc_name'
    product_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
    product_name = product_name.text
    print(f'name:{product_name}')
    return product_name

# 품목 브랜드를 가져오는 함수
def get_product_brand():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(2) > td'
    product_brand = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_brand = product_brand.text
    print(f'brand:{product_brand}')
    return product_brand

# 품목 가격을 가져오는 함수
def get_product_price():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(5) > td > span'
    product_price = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_price = product_price.text
    product_price = product_price.replace(",", '').replace(" ", "").replace("KRW", '')
    print(f'price:{product_price}')
    return product_price

# 품목 사이즈를 가져오는 함수
def get_product_size():
    id = 'prdDetail'
    product_sizes = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, id))).get_attribute("innerHTML")
    product_sizes = product_sizes.strip().split('\n')
    try:
        product_size = [x for x in product_sizes if '가슴' in x][0].split('가슴')[1].split('cm')[0].strip()
        type_ = '가슴'
    except:
        product_size = [x for x in product_sizes if '허리' in x][0].split('허리')[1].split('cm')[0].strip()
        type_ = '허리'
    print(f'type: {type_}, size:{product_size}')
    return type_, product_size

# 썸네일 이미지를 가져오는 함수
def get_product_thumbnail():
    xpath = '//*[@id="contents"]/div[1]/div[1]/div[2]/div/img'
    product_thumbnail = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath))).get_attribute('src')
    print(f'thumbnail:{product_thumbnail}')
    return product_thumbnail

# 성별 구분하는 함수
def get_product_sex():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.infoArea > div.xans-element-.xans-product.xans-product-detaildesign > table > tbody > tr:nth-child(3) > td'
    product_size = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
    product_size = product_size.text
    if product_size[0] == 'W':
        sex = 'women'
    else:
        sex = 'men'
    print(f'product size:{product_size}, {sex}')
    return sex
    
# 품목 카테고리 구분하는 함수
def get_product_category_type(product_category, sizing_name):
    if product_category == '반팔 · 반바지':
        if sizing_name == '가슴':
            category = 'top'
        elif sizing_name == '허리':
            category = 'bottom'
    elif product_category == '치마 · 원피스':
        if sizing_name == '가슴':
            category = 'dress'
        elif sizing_name == '허리':
            category = 'skirt'
    elif product_category in ['후디 · 맨투맨', '셔츠 · 긴팔티', '니트 · 가디건']:
        category = 'top'
    elif product_category in ['코트 · 아우터', '점퍼 · 스포츠', '자켓 · 블루종']:
        category = 'outer'
    else:
        category = 'others'
    print(f'category:{category}')
    return category

# 품절 여부 확인하는 함수
def is_product_sold_out():
    css = '#contents > div.xans-element-.xans-product.xans-product-detail > div.detailArea > div.headingArea > span.icon > img'
    try:
        product_sold_out = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        src = product_sold_out.get_attribute('src')
        print(f'품절 상품')
        return True
    except:
        print(f'판매중인 상품')
        return False

In [4]:
def get_clothes_info(store_name, sale_price):    
    info_dict = {}
    values = [product_name, store_name, product_category, product_gender, product_brand_after, product_price, sale_price, product_thumbnail, product_link, is_sold_out, product_size]
    keys = ['name', 'storeName', 'category', 'gender', 'brand', 'originalPrice', 'salePrice', 'thumbnailUrl', 'contentUrl', 'isSoldOut', 'size']
    if product_type == '가슴':
        product_type_ = 'chest'
    else:
        product_type_ = 'waist'
    for i in range(len(keys)):
        if keys[i] == 'size':
            info_dict[keys[i]] = {}
            info_dict[keys[i]][product_type_] = product_size 
        else:
            info_dict[keys[i]] = values[i]
    return info_dict

In [10]:
from get_brand_list import get_brand_list
driver = webdriver.Chrome(executable_path='chromedriver')
store_name = '프롬노웨어'
sale_price = '0'
for i in range(11, 17): # 바꿔
    URL = f'http://fromnowhere.co.kr/product/list.html?cate_no=165&page={i}' # 바꿔
    driver.get(url=URL)
    with open(f'{store_name}.json', 'r', encoding='utf-8') as f:
        total_list = json.load(f)
    product_list = get_product_list()
    iter = 1
    for i in range(len(product_list)):
        print(f'----------------{iter}-----------------')
        # print(product_list[i].get_attribute('innerHTML'))

        product_categories = get_product_category()
        product_link = click_product(i)
        product_name = get_product_name()
        product_brand_before = get_product_brand()
        try:
            product_price = get_product_price()
            product_type, product_size = get_product_size()
        except:
            time.sleep(0.5)
            driver.get(url=URL)
            continue
        product_thumbnail = get_product_thumbnail()
        product_gender = get_product_sex()
        product_category = get_product_category_type(product_categories, product_type)
        is_sold_out = is_product_sold_out()
        product_brand_after = get_brand_list(product_brand_before)
        json_ = get_clothes_info(store_name=store_name, sale_price=sale_price)
        total_list.append(json_)
        iter += 1
        
        time.sleep(0.5)
        driver.get(url=URL)
    with open(f'{store_name}.json', 'w', encoding='utf-8') as jsonfile:
        json.dump(total_list, jsonfile, indent=4, ensure_ascii=False)

----------------1-----------------
자켓 · 블루종
link:http://fromnowhere.co.kr/product/detail.html?product_no=111301&cate_no=165&display_group=1
name:none
brand:M
price:(T584)
type: 가슴, size:53
thumbnail:http://fromnowhere.co.kr/web/product/big/t584.jpg
product size:95, men
category:outer
품절 상품
----------------2-----------------
자켓 · 블루종
link:http://fromnowhere.co.kr/product/detail.html?product_no=111300&cate_no=165&display_group=1
name:UNIQLO
brand:L
price:(T583)
type: 가슴, size:55
thumbnail:http://fromnowhere.co.kr/web/product/big/t583.jpg
product size:105, men
category:outer
품절 상품
----------------3-----------------
자켓 · 블루종
link:http://fromnowhere.co.kr/product/detail.html?product_no=111299&cate_no=165&display_group=1
name:MLB
brand:X
price:(T582)
type: 가슴, size:64
thumbnail:http://fromnowhere.co.kr/web/product/big/t582.jpg
product size:110, men
category:outer
품절 상품
----------------4-----------------
자켓 · 블루종
link:http://fromnowhere.co.kr/product/detail.html?product_no=111289&cate_no=165&